```
```

```
pip install pymysql
pip install finance-datareader
pip install beautifulsoup4
pip install pandas
```

In [1]:
import FinanceDataReader as fdr
import pandas   as pd
import pymysql
from datetime import datetime

* MariaDB 접속

In [2]:
connection = pymysql.connect(host='localhost', port=3306, db='home', \
  user='root', passwd ='123', autocommit=True)
cursor = connection.cursor()

* fdr 에서 종목리스트를 dataframe 으로 작성

In [3]:
df_krx = fdr.StockListing('KRX')  # KRX는 KOSPI,KOSDAQ,KONEX 모두 포함
df_krx = df_krx.loc[df_krx['Code'].str.isdigit(),['Code','Name','Market']]  #테이블로 로딩할 칼럼만 선택한다.

* 종목리스트를 DB 테이블에 insert

In [6]:
count = 0
today = datetime.today().strftime('%Y%m%d')
for  idx,row in df_krx.iterrows() :
  count += 1
  
  sql = "REPLACE INTO company_info (code, name, market, last_update) VALUES (%s,%s,%s,%s)"
  cursor.execute(sql,(*row,today))
  connection.commit()
  # print(f"[company_info] {count:,} rows upsert completed")  

* 현DB에 적재된 일별가격테이블의 최종일자를 구한다.
* 최종일자 이후 일별가격만 테이블에 추가적재 하기 위함

In [7]:
sql = "select max(date) from daily_price"
cursor.execute(sql)
sdate = cursor.fetchone()[0]   # empty 이면 None  fetchone 의 return 타입은 tuple

In [8]:
if sdate is None :
	sdate = '20220101'

count=0
for idx,row in df_krx.iterrows() :
  df = fdr.DataReader(row[0], sdate)
  if df.empty :
    continue
  df = df.dropna().reset_index()  # 데이터 중에 NaN 있음, 날짜칼럼을 인텍스칼럼-> 일반칼럼
  df["Date"] = df["Date"].dt.strftime("%Y%m%d")
  for sidx,srow in df.iterrows() :
    sql = """REPLACE INTO daily_price (code, date, open, high, low, close,  volume, chg) VALUES ('{}','{}',{},{},{},{},{},{})
          """.format(row[0],*srow)
    cursor.execute(sql)
    connection.commit()
  count += 1
  print(f"[daily_price]  {count:,} upsert completed")

[daily_price]  1 upsert completed
[daily_price]  2 upsert completed
[daily_price]  3 upsert completed
[daily_price]  4 upsert completed
[daily_price]  5 upsert completed
[daily_price]  6 upsert completed
[daily_price]  7 upsert completed
[daily_price]  8 upsert completed
[daily_price]  9 upsert completed
[daily_price]  10 upsert completed
[daily_price]  11 upsert completed
[daily_price]  12 upsert completed
[daily_price]  13 upsert completed
[daily_price]  14 upsert completed
[daily_price]  15 upsert completed
[daily_price]  16 upsert completed
[daily_price]  17 upsert completed
[daily_price]  18 upsert completed
[daily_price]  19 upsert completed
[daily_price]  20 upsert completed
[daily_price]  21 upsert completed
[daily_price]  22 upsert completed
[daily_price]  23 upsert completed
[daily_price]  24 upsert completed
[daily_price]  25 upsert completed
[daily_price]  26 upsert completed
[daily_price]  27 upsert completed
[daily_price]  28 upsert completed
[daily_price]  29 upsert comp